In [1]:
import numpy as np
import json
import time
import scipy
import scipy.sparse
from trimesh import TriMesh
import glob
from numpy import *
np.set_printoptions(linewidth=2000, suppress=True)
from Extract_Transformation_matrix_minimize_SVD_singular_value import *

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#### read obj file
base_dir="./datas/cube4/"
filenames=glob.glob(base_dir+"*.obj")
print filenames
meshes_List=[]
for i in range(len(filenames)):
    meshes_List.append(TriMesh.FromOBJ_FileName(filenames[i]))

['./datas/cube4/cube.obj']


In [3]:
groundtruth_name=base_dir+"cube-2.DMAT"
gt=load_DMAT(groundtruth_name)
print gt.shape
ind=np.arange(10)
gt=gt.T
print gt.shape
gt=gt.reshape((-1,4,3))
gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
ind=np.arange(10)

IOError: [Errno 2] No such file or directory: './datas/cube4/cube-2.DMAT'

In [ ]:
#### solve for transformation per vertex, with smoothness regularization and transformation matrix L2 norm regularization.

mesh0=meshes_List[0]
mesh1=meshes_List[2]


output_prefix=base_dir+"Vertex_tramsformation_matrix_recovering"
weights = {'W_svd': 2.0, 'W_rotation': 0.01, 'W_spatial': 0.0 }


start=time.time()

print "solving transformation matrix"


vertices0=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices1=np.asarray(mesh1.vs)
M=vertices0.shape[1]*vertices1.shape[1]


x0=np.ones(len(vertices1)*M)/M
    
# x0=np.zeros(len(vertices1)*M)

# x0=gt.ravel()


res=run_one(mesh0, mesh1, output_prefix, weights, x0)
res=res.reshape((len(mesh0.vs),-1))
end=time.time()
print "using time: ", end-start
print res.shape

In [ ]:
# print abs(res[ind,:].round(3)-gt[ind,:].round(3))

print "recover: "
print res[ind,:].round(3)
print "groundtruth: "
print gt[ind,:].round(3)

vertices0=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
print "vertices reconstruction error: "
print np.multiply(res.reshape((-1,3,4)), vertices0.reshape((-1,1,4))).sum(axis=-1)[:len(ind)]-asarray(mesh1.vs)[:len(ind)]

In [ ]:
print res.shape

L=res-res.mean(axis=0).reshape((1,-1))

eigenVal,eigenVec=np.linalg.eig(L.T.dot(L))

print sorted(eigenVal)

In [ ]:
gtL=gt-gt.mean(axis=0).reshape((1,-1))

gteigenVal,gteigenVec=np.linalg.eig(gtL.T.dot(gtL))

print sorted(gteigenVal)

In [ ]:
print res.shape

L=res-res.mean(axis=0).reshape((1,-1))

s = np.linalg.svd(L, full_matrices=True, compute_uv=False)
print s.round(3)

In [ ]:
gtL=gt-gt.mean(axis=0).reshape((1,-1))

s = np.linalg.svd(gtL, full_matrices=True, compute_uv=False)
print s.round(3)

In [ ]:
temp1=res.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
identities=np.repeat(np.identity(3).reshape((1,-1)), len(temp1), 0).ravel()
print abs((temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel()-identities).max()

In [ ]:
temp1=gt.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
identities=np.repeat(np.identity(3).reshape((1,-1)), len(temp1), 0).ravel()
print abs((temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel()-identities).max()